## GO analysis on snoRNA-binding RBPs found from region calling

### Getting the RBP data from my previously generated csv file:

In [2]:
# import statements
import pandas as pd
from pathlib import Path
figdir = Path('/home/hsher/scratch/circular_fig/')

In [3]:
merged_de = pd.read_csv(figdir / 'huntington_de_circ.csv')

In [4]:
merged_de.head()

,Unnamed: 0,Case_BSJ_rep1,Case_FSJ_rep1,Case_Ratio_rep1,Ctrl_BSJ_rep1,Ctrl_FSJ_rep1,Ctrl_Ratio_rep1,DE_score_rep1,DS_score_rep1,seqname_rep1,...,DE_score_rep2,DS_score_rep2,seqname_rep2,start_rep2,end_rep2,strand_rep2,circ_type_rep2,gene_id_rep2,gene_name_rep2,gene_type_rep2
0,chr5:134790470|134811845,0,0,0.000,1,0,1.000,0.0,0.0,chr5,...,0.000000,0.0,chr5,134790470.0,134811845.0,+,exon,ENSG00000145833.16,DDX46,protein_coding
1,chr9:112268049|112297916,341,0,1.000,227,0,1.000,0.0,0.0,chr9,...,0.301168,0.0,chr9,112268049.0,112297916.0,-,exon,"ENSG00000230081.2,ENSG00000119314.16,ENSG00000...","HSPE1P28,PTBP3,RN7SL430P","processed_pseudogene,protein_coding,misc_RNA"
2,chr5:179575241|179577136,1,11,0.154,1,11,0.154,0.0,0.0,chr5,...,0.000000,0.0,chr5,179575241.0,179577136.0,+,intron,ENSG00000176783.15,RUFY1,protein_coding
3,chr17:80135313|80138280,6,0,1.000,4,0,1.000,0.0,0.0,chr17,...,0.000000,0.0,chr17,80135313.0,80138280.0,-,exon,ENSG00000141543.12,EIF4A3,protein_coding
4,chr22:20818483|20820611,1,0,1.000,2,7,0.364,0.0,0.0,chr22,...,0.000000,0.0,chr22,20818483.0,20820611.0,-,exon,ENSG00000241973.11,PI4KA,protein_coding


In [5]:
merged_de.columns

Index(['Unnamed: 0', 'Case_BSJ_rep1', 'Case_FSJ_rep1', 'Case_Ratio_rep1',
       'Ctrl_BSJ_rep1', 'Ctrl_FSJ_rep1', 'Ctrl_Ratio_rep1', 'DE_score_rep1',
       'DS_score_rep1', 'seqname_rep1', 'start_rep1', 'end_rep1',
       'strand_rep1', 'circ_type_rep1', 'gene_id_rep1', 'gene_name_rep1',
       'gene_type_rep1', 'Case_BSJ_rep2', 'Case_FSJ_rep2', 'Case_Ratio_rep2',
       'Ctrl_BSJ_rep2', 'Ctrl_FSJ_rep2', 'Ctrl_Ratio_rep2', 'DE_score_rep2',
       'DS_score_rep2', 'seqname_rep2', 'start_rep2', 'end_rep2',
       'strand_rep2', 'circ_type_rep2', 'gene_id_rep2', 'gene_name_rep2',
       'gene_type_rep2'],
      dtype='object')

In [15]:
thres = 1
upreg_circ_genes = merged_de.loc[(merged_de['DE_score_rep1']>thres)&
                                 (merged_de['DE_score_rep2']>thres)&
                                 (~merged_de['gene_name_rep2'].fillna('').str.contains(',')),
                                 'gene_name_rep2'
                                ].dropna().unique()
downreg_circ_genes = merged_de.loc[(merged_de['DE_score_rep1']<-thres)&
                                   (merged_de['DE_score_rep2']<-thres)&
                                   (~merged_de['gene_name_rep2'].fillna('').str.contains(',')),
                                   'gene_name_rep2'
                                  ].dropna().unique()
upreg_circ_genes.shape, downreg_circ_genes.shape

((246,), (279,))

In [18]:
all_genes = merged_de.loc[(~merged_de['gene_name_rep2'].fillna('').str.contains(',')),
                         'gene_name_rep2'].dropna().unique()

In [19]:
all_genes.shape

(8582,)

### From here, I follow the GO analysis tutorial:

In [20]:
# import
from goatools.obo_parser import GODag

obodag = GODag("/home/hsher/ontology/go-basic.obo")

/home/hsher/ontology/go-basic.obo: fmt(1.2) rel(2020-12-08) 47,295 Terms


In [21]:
from __future__ import print_function
from goatools.anno.genetogo_reader import Gene2GoReader

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(filename="/home/hsher/ontology/gene2go", taxids=[9606])

# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated human genes".format(NS=nspc, N=len(id2gos)))

HMS:0:00:03.044586 338,874 annotations, 20,669 genes, 18,512 GOs, 1 taxids READ: /home/hsher/ontology/gene2go 
BP 18,669 annotated human genes
MF 18,166 annotated human genes
CC 19,379 annotated human genes


In [22]:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

In [23]:
import sys
sys.path.append('/home/hsher/ontology')
from protein_coding_9606 import GENEID2NT as human_genes

### getting the snoRNA-binding RBPs (background) and the list to be compared to
My list of snoRNA-binding RBPs (the background) already exists as `snoRNA_rbp`. 
My comparison list (the list of RBPs with a threshold of how many snoRNAs they bind) exists as `snoRNA_rbp_K562_200`. 

In [24]:
import mygene
mg = mygene.MyGeneInfo()
mgresult = mg.querymany(all_genes, scopes='symbol', fields='entrezgene', as_dataframe = True)
mgresult.dropna(subset = ['entrezgene'], inplace = True)
mgresult['entrezgene']=mgresult['entrezgene'].astype(int)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-8582...done.
Finished.
7857 input query terms found dup hits:
	[('DDX46', 10), ('RUFY1', 10), ('EIF4A3', 10), ('PI4KA', 10), ('GLB1', 10), ('TTLL7', 10), ('ARHGAP2
515 input query terms found no hit:
	['AC023421.2', 'AL589740.1', 'Z93929.2', 'AC111006.1', 'AL008633.1', 'AL158212.1', 'AC068282.1', 'AC
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [25]:
gene_ids=list(set(human_genes.keys()).intersection(set(mgresult['entrezgene'].tolist())))

In [27]:
# first I check if any of them don't have a entrezgene id
len(set(all_genes)-set(mgresult.index)) # these are not in the mygene database

537

In [28]:
# some are not in the 'protein_coding_9606' from the GOATTOOLS package 'human_gene'
mgresult.loc[~mgresult['entrezgene'].isin(human_genes.keys())]

,_id,_score,entrezgene,notfound
query,,,,
DDX46,212880,14.686645,212880,NaN
DDX46,245957,12.427162,245957,NaN
DDX46,462070,11.297420,462070,NaN
DDX46,116056045,11.297420,116056045,NaN
DDX46,111659968,11.297420,111659968,NaN
...,...,...,...,...
DCAF8L2,702202,14.896118,702202,NaN
DCAF8L2,105574594,14.896118,105574594,NaN
DCAF8L2,101151703,14.896118,101151703,NaN


In [31]:
len(all_genes) # we lost 10 genes

8582

In [32]:
id_mapper = mgresult.loc[mgresult['entrezgene'].astype(int).isin(gene_ids), 'entrezgene'
                        ].to_dict()

In [34]:
# import os

from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj
# outdir='/home/hsher/covid_rmats_as_rep/GO/'

def main(protein_list, background, mapper = id_mapper):
    """ Charlene's code. 
    protein_list: gene symbols of RBP
    background: ENTREZGENE ID of the background, therefore we use id_mapper.values()
    
    """
    
    # start enrichment analysis
    goeaobj = GOEnrichmentStudyNS(
        background, # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method

    geneid=[id_mapper[rbp] for rbp in protein_list]
    

    goea_results_all = goeaobj.run_study(geneid)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


    #goeaobj.wr_xlsx(outdir+"{}_{}_{}.xlsx".format(protein, type_event, di), goea_results_sig)
    #plot_results(outdir+"{}_{}_{}".format(protein, type_event, di)+"_{NS}.png", goea_results_sig)
    
    return goea_results_all

In [35]:
# we will have to omit the protein, both in background and in the query that failed to find it's ID in the GOATTOOL package
go_output = main(list(set(upreg_circ_genes).intersection(id_mapper.keys())),
                id_mapper.values())
                
# commented out because of weird error 
# it is because GSK3 is not in id_mapper
# we should check why it is not there


Load BP Ontology Enrichment Analysis ...
 92%  6,887 of  7,462 population items found in association

Load CC Ontology Enrichment Analysis ...
 96%  7,130 of  7,462 population items found in association

Load MF Ontology Enrichment Analysis ...
 94%  7,013 of  7,462 population items found in association

Runing BP Ontology Analysis: current study set of 223 IDs.
 96%    214 of    223 study items found in association
100%    223 of    223 study items found in population(7462)
Calculating 9,009 uncorrected p-values using fisher_scipy_stats
   9,009 terms are associated with  6,887 of  7,462 population items
   1,252 terms are associated with    214 of    223 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing CC Ontology Analysis: current study set of 223 IDs.
 97%    21

In [36]:
[[r.name for r in go_output if r.p_fdr_bh < 0.2]]

[[]]

In [37]:
# we will have to omit the protein, both in background and in the query that failed to find it's ID in the GOATTOOL package
go_down_output = main(list(set(downreg_circ_genes).intersection(id_mapper.keys())),
                id_mapper.values())
                
# commented out because of weird error 
# it is because GSK3 is not in id_mapper
# we should check why it is not there


Load BP Ontology Enrichment Analysis ...
 92%  6,887 of  7,462 population items found in association

Load CC Ontology Enrichment Analysis ...
 96%  7,130 of  7,462 population items found in association

Load MF Ontology Enrichment Analysis ...
 94%  7,013 of  7,462 population items found in association

Runing BP Ontology Analysis: current study set of 257 IDs.
 94%    241 of    257 study items found in association
100%    257 of    257 study items found in population(7462)
Calculating 9,009 uncorrected p-values using fisher_scipy_stats
   9,009 terms are associated with  6,887 of  7,462 population items
   1,292 terms are associated with    241 of    257 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Runing CC Ontology Analysis: current study set of 257 IDs.
 96%    24

In [38]:
# see what is significant
[r.name for r in go_down_output if r.p_fdr_bh < 0.05]

[]